In [1]:
println("Number of threads that your Julia is run: ## $(Threads.nthreads())")

Number of threads that your Julia is run: ## 8


In [3]:
using BenchmarkTools, Distributed

### Parallelize via Single Instruction Multiple Data (SIMD)

In [4]:
function dot1(x, y)
    s = 0.0
    for i in 1:length(x)
        @inbounds s += x[i]*y[i]
    end
    s
end

dot1 (generic function with 1 method)

In [5]:
function dot2(x, y)
    s = 0.0
    @simd for i in 1:length(x)
        @inbounds s += x[i]*y[i]
    end
    s
end

dot2 (generic function with 1 method)

In [6]:
x = 100*rand(10000)
y = 100*rand(10000)

res1 = @btime dot1($x, $y)
res2 = @btime dot2($x, $y)

println(res1)
println(res2)

  12.025 μs (0 allocations: 0 bytes)
  2.110 μs (0 allocations: 0 bytes)
2.5095465536428425e7
2.5095465536428474e7


### Green threading

In [9]:
@time sleep(2)

  2.005748 seconds (64 allocations: 1.922 KiB)


In [10]:
@time @async sleep(2)

  0.005964 seconds (8.01 k allocations: 462.538 KiB)


Task (runnable) @0x0000023ddb060200

In [24]:
function dojob(i)
    val = round(rand(), digits=2)
    sleep(val)   # this could be external computations with I/O
    i, val
end

dojob (generic function with 1 method)

In [25]:
result = Vector{Tuple{Int,Float64}}(undef, 8)

8-element Vector{Tuple{Int64, Float64}}:
 (0, 0.0)
 (0, 0.0)
 (0, 0.0)
 (0, 0.0)
 (0, 0.0)
 (0, 0.0)
 (0, 0.0)
 (0, 0.0)

In [26]:
@time for i=1:8
    result[i] = dojob(i)
end
result

  5.950367 seconds (213 allocations: 6.172 KiB)


8-element Vector{Tuple{Int64, Float64}}:
 (1, 0.91)
 (2, 0.99)
 (3, 0.96)
 (4, 0.4)
 (5, 0.48)
 (6, 0.99)
 (7, 0.54)
 (8, 0.62)

In [23]:
result = Vector{Tuple{Int,Float64}}(undef, 8);
@time for i=1:8
   @async result[i] = dojob(i)
end
result

  0.000062 seconds (83 allocations: 7.139 KiB)


8-element Vector{Tuple{Int64, Float64}}:
 (0, 0.0)
 (0, 0.0)
 (0, 0.0)
 (0, 0.0)
 (0, 0.0)
 (0, 0.0)
 (0, 0.0)
 (0, 0.0)

In [22]:
result = Vector{Tuple{Int,Float64}}(undef, 8);
@time @sync for i=1:8
   @async result[i] = dojob(i)
end
result

  0.936859 seconds (6.78 k allocations: 420.356 KiB, 1.18% compilation time)


8-element Vector{Tuple{Int64, Float64}}:
 (1, 0.15702367133816209)
 (2, 0.9073111610283118)
 (3, 0.10898634791280126)
 (4, 0.09863688214715938)
 (5, 0.07279478059884814)
 (6, 0.5107022859229167)
 (7, 0.3750897195590063)
 (8, 0.44310129982972624)

#### Programming a simple web server
You should be able to connect using the address <a href="http://localhost:9991/3+4" target="about:blank">http://localhost:9991/3+4</a>

To stop web server click <a href="http://localhost:9991/stopme" target="about:blank">http://localhost:9991/stopme</a>

In [ ]:
using Sockets
println("Starting the web server...")
server = Sockets.listen(9991)

In [7]:
contt = Ref(true)
while contt[]
    sock = Sockets.accept(server)
    @async begin
        data = readline(sock)
        print("Got request:\n", data, "\n")
        cmd = split(data, " ")[2][2:end]
        println(sock, "\nHTTP/1.1 200 OK\nContent-Type: text/html\n")
        contt[] = contt[] && (!occursin("stopme", data))
        if contt[]
             println(sock, string("<html><body>", cmd, "=", eval(Meta.parse(cmd)), "</body></html>"))
        else
            println(sock,"<html><body>stopping</body></html>")
        end
        close(sock)
    end
end
println("Server stopped")


Got request:
GET /stopme HTTP/1.1
Server stopped
Got request:



### Multithreading

In [27]:
Threads.nthreads()

8

In [28]:
function ssum(x)
    r, c = size(x)
    y = zeros(c)
    for i in 1:c
        for j in 1:r
            @inbounds y[i] += x[j, i]
        end
    end
    y
end

ssum (generic function with 1 method)

In [30]:
function tsum(x)
    r, c = size(x)
    y = zeros(c)
    Threads.@threads for i in 1:c
        for j in 1:r
            @inbounds y[i] += x[j, i]
        end
    end
    y
end


tsum (generic function with 1 method)

In [42]:
x = rand(10000,1000);

In [43]:
@time ssum(x)
@time ssum(x);

  0.015025 seconds (1 allocation: 7.938 KiB)
  0.009474 seconds (1 allocation: 7.938 KiB)


In [44]:
@time tsum(x)
@time tsum(x);

  0.004882 seconds (52 allocations: 13.359 KiB)
  0.001982 seconds (52 allocations: 13.359 KiB)


#### Locking mechanism for threads

In [45]:
function f_bad()
    x = 0
    Threads.@threads for i in 1:10^7
        x += 1
    end
    return x
end


f_bad (generic function with 1 method)

In [48]:
@time f_bad()

  0.525740 seconds (10.00 M allocations: 152.578 MiB)


1293224

In [49]:
function f_atomic()
    x = Threads.Atomic{Int}(0)
    Threads.@threads for i in 1:10^7
        Threads.atomic_add!(x, 1)
    end
    return x[]
end

function f_spin()
    l = Threads.SpinLock()
    x = 0
    Threads.@threads for i in 1:10^7
        Threads.lock(l) do
            x += 1
        end
    end
    return x
end

function f_reentrant()
    l = ReentrantLock()
    x = 0
    Threads.@threads for i in 1:10^7
        Threads.lock(l) do
            x += 1
        end
    end
    return x
end


f_reentrant (generic function with 1 method)

In [50]:
using DataFrames
stats = DataFrame()
for f in [f_bad, f_atomic, f_spin, f_reentrant]
    for i = 1:2
        value, elapsedtime  = @timed f()
        push!(stats, (f=string(f),i=i, value=value, timems=elapsedtime*1000))
    end
end
println(stats)


8×4 DataFrame
 Row │ f            i      value     timems   
     │ String       Int64  Int64     Float64  
─────┼────────────────────────────────────────
   1 │ f_bad            1   1306731   254.793
   2 │ f_bad            2   1494738   397.46
   3 │ f_atomic         1  10000000   231.957
   4 │ f_atomic         2  10000000   195.303
   5 │ f_spin           1  10000000  4758.85
   6 │ f_spin           2  10000000  4721.66
   7 │ f_reentrant      1  10000000  6788.93
   8 │ f_reentrant      2  10000000  6573.98


### Multi-processing and distributed computing

In [51]:
using Distributed

In [61]:
addprocs(max(0, 5-nworkers()));

In [62]:
workers()

5-element Vector{Int64}:
 2
 3
 4
 5
 6

In [63]:
function s_rand()
    n = 10^4
    x = 0.0
    for i in 1:n
        x += sum(rand(10^4))
    end
    x / n
end
 
@time s_rand()
@time s_rand()


  0.435935 seconds (20.00 k allocations: 763.397 MiB, 22.88% gc time)
  0.419023 seconds (20.00 k allocations: 763.397 MiB, 16.58% gc time)


5000.002440794839

In [64]:
using Distributed
 
 
function p_rand()
    n = 10^4
    x = @distributed (+) for i in 1:n
        sum(rand(10^4))
    end
    x / n
end

@time p_rand()
@time p_rand()


  1.312609 seconds (1.01 M allocations: 51.940 MiB, 40.42% compilation time)
  0.133961 seconds (488 allocations: 25.766 KiB)


4999.590348317194